# Knowledge Distillation For Fine-Tuning A GPT-3.5 Judge

There has been recent research that demonstrated GPT-4's ability to closely align to human judges when evaluating LLM generated texts (e.g., see [[1]](https://arxiv.org/abs/2306.05685), [[2]](https://arxiv.org/abs/2303.16634)). In this notebook, we demonstrate how to use the `llama_index` library to distill knowledge from GPT-4 to GPT-3.5 so that the smaller GPT-3.5 becomes closer to GPT-4 performance; and by proxy, closer to human judges.

To do so, we take the following steps:

1. Generate datasets: `train` and `test`
2. Perform knowledge distillation (using `train`)
3. Evaluate the distilled model  on `test`

## 1 Generate datasets: `train` and `test`

We should not lose sight on the ultimate goal here, which is to build an LLM judge that closely matches to human judges when evaluating LLM-generated texts. The work we need to do in this step, therefore, is to build a set of generated texts that our LLM judges will judge. More specifically, we will follow the "single-grading" evaluation design pattern, where one text generation is passed to an LLM judge that is subsequently prompted to assign a score between 0 and 1 (higher is better).

To generate a varied set of texts we'll use the following LLM text-generators:
1. HuggingFace: Vicuna-13B
2. HuggingFace: Mistral-7B
3. HuggingFace: Falcon-7B

The generation task we ask of each of these models will be to generate an abstractive answer to question when provided relevant context (i.e., RAG).

### Using `DatasetGenerator` to build `train` and `test`

The specific procedure we will use here involves generating questions against a set of chunks of a given `Document`. With the `<question, chunk>` pairs in hand, (for which we can merely treat as a "simulated" retrieval), we pass this information to the three LLM generators and prompt them each to generate an answer.

Hang tight, we're almost there (sort of). It's important to note now that our learning objective for performing knowledge distillation takes on the following form:
$$
\mathcal{L}_{student} = \alpha\mathcal{L}_{CE} + (1-\alpha)\mathcal{L}_{KD},
$$
where $\mathcal{L}_{CE}$ is the usual cross-entropy loss and $\mathcal{L}_{KD}$ is the knowledge-distillation loss (which if you're keen on knowing will be based on the Kullback-Leibler divergence).

Computing $\mathcal{L}_{KD}$ in our case requires us to actually get knowledge from the teacher model (i.e., GPT-4). To do that, we will need to prompt GPT-4 to judge a set of generated answers. Specifically, we will present the GPT-4 judge with a single LLM-generated answers and prompt it to assign a score, ranging between 0 and 1, to it. To turn this into a classification problem, we will now assert than any score greater than 0.8 to be an "acceptable" answer, and an "unacceptable" one otherwise. 

With all of that we can now build a `dataset` that looks like the one below.
| question | generated-answer | gpt-4-score | gpt-4-classification |
|----------|------------------|-------------|----------------------|
| ...      | ...              | ...         | ...                  |

And finally, to get `train` and `test` we will simply randomly shuffle `dataset` and split it using a 70/30 ratio. (Phew!)

In [ ]:
from llama_index import SimpleDirectoryReader, ServiceContext

# load a document

# split document into chunks

# generate questions against chunks

In [ ]:
from llama_index.llms import HuggingFaceLLM, OpenAI

# define our llm-generators

# define our llm judges (also student/teacher models)

In [ ]:
# create our dataset, and split into train and test

## 2 Perform knowledge distillation

Okay, it's now time to distill some knowledge from GPT-4 to GPT-3.5 To do this, we will make use of `OpenAIFinetuneEngine` class of `llama_index`. 